1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [1]:
HexL=['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f']

#b_in and b_out are the basis from which and to which i want to convert
#y is the output number in the asked base

def convert(n,b_out):
    x=(str)(n)
    #check the input to detect the input base
    if x[0:2]=='0b':
        x=x[2:]
        b_in=2
    elif x[0:2]=='0x':
        x=x[2:]
        b_in=16
    else:
        b_in=10
    n2=0
    l=len(x)
    for i, j in enumerate(x):
        for a, b in enumerate(HexL):
            if j==b:
                n2+=a*(b_in)**(l-(i+1))
    y=''
    while n2>0:
        r=n2%b_out #resto
        y=HexL[(int)(r)]+y
        n2=(n2-r)/b_out
    print(y)
    
convert(24,2) #dec to bin
convert(24,16) #dec to hex
convert('0b11000',10) #bin to dec
convert('0b11000',16) #bin to hex
convert('0x18',2) #hex to bin
convert('0x18',10) #hex to dec
#we can also try to convert to the same base of the input and it will work returning the same number given in input, but it doesn't make sense

11000
18
24
18
11000
24


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [2]:
e=127 #offset

def tofloat32(num):
    #firts we do the & operation with the masks
    sign = (num & 0x80000000) >> 31 #moving sign to the firts bit
    exp = (num & 0x7F800000) >> 23 #moving exp to the first 8 bits
    mantissa = (num & 0x7FFFFF) #no need to shift
    base = 1
    for i in range(1,23):
        base +=(2.0**(-i))*((mantissa >> (23 - i))&0x1)
    temp = base*(2.0**(exp-e))
    #check sign to return
    if sign == 1:
        return -temp
    return temp
            
tofloat32(0x40404040) #it should be around 3.00

3.0039215087890625

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [3]:
import math

a=1
c=0

#if we divide by 2 1075 times we get 0
while a!=0: 
    c=c+1
    a=a/2.0
print("Underflow halved ",c, " times.")

x=1
#dividing by 2 1074 times the value is still acceptable
for i in range (1074): 
    x=x/2.0
print("Underflow: ",x)

b=1
c2=0

#if we multiply by 2 1024 times we get inf
while b!=math.inf: 
    c2=c2+1
    b=b*2.0
print("Overflow doubled ",c2, " times.")

y=1
#multipling by 2 1023 times the value is still acceptable
for j in range (1023): 
    y=y*2.0
print("Overflow: ",y)

Underflow halved  1075  times.
Underflow:  5e-324
Overflow doubled  1024  times.
Overflow:  8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [4]:
import sys
import numpy as np

#using numpy
print("With numpy: ",np.finfo(float).eps)

#using sys
epsilon=sys.float_info.epsilon
print("With sys: ",epsilon)

#third mode without modules
n = 1.
eps = 1.

while n != n + eps:
    eps = eps / 2

eps = eps * 2  

print("Third mode: ",eps)
#we can see that in the manually computed result is equal to the results given using the modules

With numpy:  2.220446049250313e-16
With sys:  2.220446049250313e-16
Third mode:  2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [5]:
import math

def quadratic(a,b,c):
    x1=(-b+math.sqrt(b**2-4*a*c))/(2*a)
    x2=(-b-math.sqrt(b**2-4*a*c))/(2*a)
    s=[x1,x2]
    return s
print('Case a: ',quadratic(0.001,1000,0.001))

def alternative(a,b,c):
    x1=(4*a*c)/(-2*a*b-2*a*math.sqrt(b**2-4*a*c))
    x2=(4*a*c)/(-2*a*b+2*a*math.sqrt(b**2-4*a*c))
    s=[x1,x2]
    return s
print('Case b: ',alternative(0.001,1000,0.001))

def correct(a,b,c):
    x1=(4*a*c)/(-2*a*b-2*a*math.sqrt(b**2-4*a*c))
    x2=(-b-math.sqrt(b**2-4*a*c))/(2*a)
    s=[x1,x2]
    return s
print('Case c: ',correct(0.001,1000,0.001))

Case a:  [-9.999894245993346e-07, -999999.999999]
Case b:  [-1.0000000000009999e-06, -1000022.1222095027]
Case c:  [-1.0000000000009999e-06, -999999.999999]


#### Comment to ex5
We can have problems when summing 'b' and the result of the discriminant.
We can notice that when 'b' is very big with respect to '4ac', then the discriminant is very close to 'b' and its difference with 'b' becomes a problem. But this happens only in one case. In fact in case the difference is evaluated, it gives troubles. Instead, when the sum is evaluated, it does not create problems.

At the end, for every pair of solutions only one gives the correct result
To overcome this problem we can use a pair of solutions like above, which is case sensitive with respect to the sign of 'b'.

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [6]:
def f(x):
    return x*(x-1)

x = 1
delta = 1e-2
derivative = (f(x+delta) - f(x)) / delta

print("Derivative: ",derivative, "\n")

for exp in range(2, 15, 2):
    delta = 10**(-exp)
    derivative = (f(x+delta) - f(x)) / delta
    print("Delta = ", delta, ". Derivative: ", derivative, ", error: ", 1-derivative)

Derivative:  1.010000000000001 

Delta =  0.01 . Derivative:  1.010000000000001 , error:  -0.010000000000000897
Delta =  0.0001 . Derivative:  1.0000999999998899 , error:  -9.999999988985486e-05
Delta =  1e-06 . Derivative:  1.0000009999177333 , error:  -9.99917733279787e-07
Delta =  1e-08 . Derivative:  1.0000000039225287 , error:  -3.922528746258536e-09
Delta =  1e-10 . Derivative:  1.000000082840371 , error:  -8.284037100736441e-08
Delta =  1e-12 . Derivative:  1.0000889005833413 , error:  -8.890058334132256e-05
Delta =  1e-14 . Derivative:  0.9992007221626509 , error:  0.0007992778373491216


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [7]:
import math

def function(x):
    return math.sqrt(1-x*x)

def integral(f, a, b, N):
    h = (b-a)/N
    x = a
    I = 0
    for k in range(N+1):
        try:
            I += f(x)
            x += h
        except:
            break
    return I*h

#N=100
%time I100 = integral(function, -1,1,100)
print("Integral with N=100:", I100 )
print("Error with N=100:", I100 - math.pi/2 )

#less than 1 second of running increasing N
%time Isecond = integral(function, -1,1,3300000)
print("Integral with less than 1 second running:", Isecond )
print("Error with less than 1 second running:", Isecond - math.pi/2 )

#about 1 minute of running increasing N
%time Iminute = integral(function, -1,1,220000000)
print("Integral with about 1 minute running:", Iminute )
print("Error with about 1 minute running:", Iminute - math.pi/2 )

CPU times: user 49 µs, sys: 1 µs, total: 50 µs
Wall time: 55.1 µs
Integral with N=100: 1.5691342555492493
Error with N=100: -0.001662071245647212
CPU times: user 800 ms, sys: 27 ms, total: 827 ms
Wall time: 865 ms
Integral with about 1 second running: 1.5707963265699172
Error with about 1 second running: -2.2497936846832545e-10
CPU times: user 53.7 s, sys: 1.01 s, total: 54.8 s
Wall time: 56.2 s
Integral with about 1 minute running: 1.5707963244364498
Error with about 1 minute running: -2.35844677121122e-09
